In [4]:
from sdv import Metadata
import os

print(os.getcwd())
from test_20_tables import fetch_data_from_sqlite_filter

metadata, tables = fetch_data_from_sqlite_filter(["BookLoan",
                 "Book", "Library", "Student",
                 "Enrollment", "Submission", "Course", # "Assignment"
                 "CourseTextbook", "Textbook",
                 "Schedule", "Professor"],"1k_data_sqlite.db")
metadata = Metadata(metadata)
metadata

C:\Users\Elvin\Desktop\GitHub\SDV\tutorials\relational_data


Metadata
  root_path: .
  tables: ['BookLoan', 'Book', 'Library', 'Student', 'Enrollment', 'Submission', 'Course', 'CourseTextbook', 'Textbook', 'Schedule', 'Professor']
  relationships:
    BookLoan.book_id -> Book.book_id
    BookLoan.student_id -> Student.student_id
    Book.library_id -> Library.library_id
    Enrollment.student_id -> Student.student_id
    Submission.student_id -> Student.student_id
    CourseTextbook.textbook_id -> Textbook.textbook_id
    Schedule.professor_id -> Professor.professor_id

In [ ]:
metadata.to_json("1.json")

In [ ]:
for key,table in tables.items():
    print(table.head(1))

In [ ]:
from sdv.relational import HMA1
import warnings
warnings.filterwarnings('ignore')

model = HMA1(metadata)    

In [ ]:
model.fit(tables)

In [ ]:
model.save('my_model_1k.pkl')

In [ ]:
new_data = model.sample(num_rows=1000)
new_data

In [ ]:
synthetic_data = new_data
real_data = tables

from sdv.evaluation import evaluate
evaluate(synthetic_data, real_data,metadata=metadata, aggregate=False)

In [ ]:
evaluate(synthetic_data, real_data,metadata=metadata)

In [2]:
o_real_ctgan_data = real_ctgan_data.copy()

In [3]:
for key, table in o_real_ctgan_data.items():
    real_ctgan_data[key] = table.sample(30000) #[0:1000]
    print(len(table))

ValueError: Cannot take a larger sample than population when 'replace=False'

In [4]:
# from sdv.evaluation import evaluate
# ret = evaluate(synthetic_ctgan_data, real_ctgan_data, metadata=metadata)
# ret

In [36]:
# real_ctgan_data["Book"]

In [9]:
import pandas as pd

def read_xlsx(name):
    # 读取多sheet的Excel文件
    sheets_dict = pd.read_excel(name, sheet_name=None)
    return sheets_dict
synthetic_ctgan_data = read_xlsx("test_100k_sample_split.xlsx")
real_ctgan_data = read_xlsx("test_100k_origin_split.xlsx")
import pickle
from sdv.evaluation import evaluate
metadata = pickle.load(open("test_data.pickle", "rb"))
ret = evaluate(synthetic_ctgan_data, real_ctgan_data,metadata=metadata, aggregate=False)
print(ret.normalized_score.mean())
ret

0.7489928639741503


,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.816659,0.816659,0.0,1.0,MAXIMIZE,None
1,CSTest,Chi-Squared,0.681326,0.681326,0.0,1.0,MAXIMIZE,None
